In [3]:
import os
import sys

current_dir = os.getcwd()
module_path = os.path.abspath(os.path.join(current_dir, '..', 'tokenizer'))
sys.path.append(module_path)

from tokenizerv2 import TokenizerV2
tokenizer = TokenizerV2()
tokenizer.load_object('../tokenizer.pkl')

In [12]:
from datasets import load_dataset
from tqdm import tqdm

ds = load_dataset("stanfordnlp/imdb")

train = ds['train']
test = ds['test']

# Batch processing for train data with a progress bar
X_train = [tokenizer.encode(text) for text in tqdm(train['text'], desc="Encoding train data")]
y_train = train['label']





Encoding train data: 100%|██████████| 25000/25000 [00:15<00:00, 1650.40it/s]


In [17]:
import numpy as np
def td_idf(X):
    
    
    
    return tf_idf

In [18]:
td_idf_train = td_idf(X_train)

NameError: name 'tf_idf' is not defined